In [ ]:
import sys
!{sys.executable} -m pip install plotly

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import umap

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext

import time 
from time import sleep

sc = SparkContext()
sc

In [ ]:
spark = SparkSession \
 .builder \
 .appName("Python Spark SQL basic example") \
 .config("spark.some.config.option", "some-value") \
 .getOrCreate()

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType,ArrayType

#### Read in Embedding 

In [ ]:
result = spark.read.csv('s3://s3://hids511final/corid_embedding_pca_200.parquet',header = True)

In [ ]:
sample_seed_1 = result[['_c0','EMBEDDING','pca_features']].sample(.01,1)

In [ ]:
df = pd.DataFrame(sample_seed_1[['_c0','EMBEDDING']].collect())

In [ ]:
X = df['EMBEDDING'].values

#### Umap

In [ ]:

umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=5, 
                            metric='cosine').fit_transform(embeddings)

In [ ]:
import hdbscan
cluster = hdbscan.HDBSCAN(min_cluster_size=20,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)

In [ ]:
import matplotlib.pyplot as plt

# Prepare data
umap_data = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
result = pd.DataFrame(umap_data, columns=['x', 'y'])
result['labels'] = cluster.labels_

# Visualize clusters
fig, ax = plt.subplots(figsize=(20, 10))
outliers = result.loc[result.labels == -1, :]
clustered = result.loc[result.labels != -1, :]
plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=0.05)
plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=0.05, cmap='hsv_r')
plt.colorbar()